<a href="https://colab.research.google.com/github/isabellacadisco/Algorithms-for-Massive-Data-course/blob/main/LSH2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
# download dataset from kaggle and save it in drive


#import os
os.environ['KAGGLE_USERNAME'] = "isabellacadisco"
os.environ['KAGGLE_KEY'] = "847325691c2399e41dc9c0b78fbe499e"
!kaggle datasets download -d xhlulu/medal-emnlp

100% 6.81G/6.82G [01:24<00:00, 57.2MB/s]
100% 6.82G/6.82G [01:24<00:00, 86.6MB/s]


In [ ]:
!unzip medal-emnlp.zip #unzip dataset

Archive:  medal-emnlp.zip
  inflating: full_data.csv           
  inflating: pretrain_subset/test.csv  
  inflating: pretrain_subset/train.csv  
  inflating: pretrain_subset/valid.csv  


In [ ]:
# set up spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #install java as VM
!wget -q https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz #download Spark
!tar xf spark-3.4.0-bin-hadoop3.tgz #unzip the file
!pip install -q findspark

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

In [ ]:
import findspark
findspark.init("spark-3.4.0-bin-hadoop3")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate() #SPARK SESSION

In [ ]:
import pyspark
type(spark)

pyspark.sql.session.SparkSession

In [ ]:
file_path = './full_data.csv'

In [ ]:
data_df = spark.read.csv(file_path, header=True)

In [ ]:
data_df.show()

+--------------------+--------------------+--------------------+
|                TEXT|            LOCATION|               LABEL|
+--------------------+--------------------+--------------------+
|alphabisabolol ha...|                  56|           substrate|
|a report is given...|24|49|68|113|137|172|carcinosarcoma|re...|
|the virostatic co...|                  55|           substrate|
|rmi rmi and rmi a...|   25|82|127|182|222|compounds|compoun...|
|a doubleblind stu...|22|26|28|77|90|14...|oxazepam|placebo|...|
|stroma from eithe...|         6|82|84|107|red cells|serum|a...|
|the effect of the...|                4|13|major|pentose pho...|
|in one experiment...|        32|44|76|135|feeding|feeding|a...|
|the presence of a...|7|15|63|137|199|2...|active|study|acti...|
|the reaction of g...|     113|203|209|250|stable|assay|bind...|
|choline acetyltra...|                  44|             caudate|
|increasing concen...|                  81|        displacement|
|the properties of...|   

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

In [ ]:
dataset = data_df.select('TEXT').withColumn('doc_id', monotonically_increasing_id())

In [ ]:
dataset.show()

+--------------------+------+
|                TEXT|doc_id|
+--------------------+------+
|alphabisabolol ha...|     0|
|a report is given...|     1|
|the virostatic co...|     2|
|rmi rmi and rmi a...|     3|
|a doubleblind stu...|     4|
|stroma from eithe...|     5|
|the effect of the...|     6|
|in one experiment...|     7|
|the presence of a...|     8|
|the reaction of g...|     9|
|choline acetyltra...|    10|
|increasing concen...|    11|
|the properties of...|    12|
|primary amines re...|    13|
|a purification pr...|    14|
|dihydrofolate red...|    15|
|ionization effect...|    16|
|kinetic analyses ...|    17|
|the nearultraviol...|    18|
|the circular pola...|    19|
+--------------------+------+
only showing top 20 rows



In [ ]:
dataset_reduced = dataset.limit(1000)

In [ ]:
dataset_reduced.show()

+--------------------+------+
|                TEXT|doc_id|
+--------------------+------+
|alphabisabolol ha...|     0|
|a report is given...|     1|
|the virostatic co...|     2|
|rmi rmi and rmi a...|     3|
|a doubleblind stu...|     4|
|stroma from eithe...|     5|
|the effect of the...|     6|
|in one experiment...|     7|
|the presence of a...|     8|
|the reaction of g...|     9|
|choline acetyltra...|    10|
|increasing concen...|    11|
|the properties of...|    12|
|primary amines re...|    13|
|a purification pr...|    14|
|dihydrofolate red...|    15|
|ionization effect...|    16|
|kinetic analyses ...|    17|
|the nearultraviol...|    18|
|the circular pola...|    19|
+--------------------+------+
only showing top 20 rows



# Class Hash Family

In [ ]:
# ci sono degli import che non ho eseguito ma ho potuto comunque creare classe
# forse sono già in colab boh

import re, hashlib, math, time
from random import randint, seed

# però sono abbastanza certa che il seed mi serva

seed(16)

In [ ]:
# HO MESSO BYTE RITORNO A 4, PENSO 2^32 BUCKET THEN
# BUT WHO KNOWS

class hashFamily:
    def __init__(self, i):
        self.resultSize = 4 # how many bytes we want back
        self.maxLen = 20 # how long can our i be (in decimal)
        self.salt = str(i).zfill(self.maxLen)[-self.maxLen:]
        self.id = i

    def get_hash_value(self, el_to_hash):
        return int(hashlib.sha1(str(el_to_hash).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)


# Class Shingler

In [ ]:
class shingler:
    def __init__(self, k):

        if k > 0:
            self.k = int(k)
        else:
            self.k = 10

    #inner class utility
    def process_doc(self, document):
        return re.sub("( )+|(\n)+"," ",document).lower()

    def get_shingles(self, document):
        shingles = set()
        document= self.process_doc(document)
        for i in range(0, len(document)-self.k+1 ):
            shingles.add(document[i:i+self.k])
        return shingles

    def get_k(self):
        return self.k

    #return sorted hash
    def get_hashed_shingles(self, shingles_set):
        hash_function = hashFamily(0)
        return sorted( {hash_function.get_hash_value(s) for s in shingles_set} )

# Shingling

In [ ]:
def shingling_map(row):
    out = []
    sh_instance = shingler(10)
    hashed_shingles = sh_instance.get_hashed_shingles( sh_instance.get_shingles( row['TEXT'] ) )
    signature_size = 50
    for i in range(0,signature_size): #signature size
        out.append( (row['doc_id'], hashed_shingles, i ) )
    #return an iterator to use flatMap => produce more than one key-value pair as output (namely one per hash function)
    return iter(out)




In [ ]:
docId_shingleset_hfunc = dataset_reduced.rdd.flatMap(shingling_map)

In [ ]:
docId_shingleset_hfunc.toDF().show()

+---+--------------------+---+
| _1|                  _2| _3|
+---+--------------------+---+
|  0|[26, 682, 687, 88...|  0|
|  0|[26, 682, 687, 88...|  1|
|  0|[26, 682, 687, 88...|  2|
|  0|[26, 682, 687, 88...|  3|
|  0|[26, 682, 687, 88...|  4|
|  0|[26, 682, 687, 88...|  5|
|  0|[26, 682, 687, 88...|  6|
|  0|[26, 682, 687, 88...|  7|
|  0|[26, 682, 687, 88...|  8|
|  0|[26, 682, 687, 88...|  9|
|  0|[26, 682, 687, 88...| 10|
|  0|[26, 682, 687, 88...| 11|
|  0|[26, 682, 687, 88...| 12|
|  0|[26, 682, 687, 88...| 13|
|  0|[26, 682, 687, 88...| 14|
|  0|[26, 682, 687, 88...| 15|
|  0|[26, 682, 687, 88...| 16|
|  0|[26, 682, 687, 88...| 17|
|  0|[26, 682, 687, 88...| 18|
|  0|[26, 682, 687, 88...| 19|
+---+--------------------+---+
only showing top 20 rows



# Min Hashing

In [ ]:
def minhash_map(docId_ShingleSet_hFunct):
    doc_id = docId_ShingleSet_hFunct[0]
    shingles = docId_ShingleSet_hFunct[1]
    hash_f = hashFamily( docId_ShingleSet_hFunct[2] )
    min_h = math.inf
    for el in shingles:
        hash_value = hash_f.get_hash_value(el)
        if hash_value < min_h:
            min_h = hash_value

    return (doc_id, min_h )

In [ ]:
sig_matrix = docId_shingleset_hfunc.map(minhash_map).groupByKey().map(lambda x : (x[0], list(x[1])))

In [ ]:
sig_matrix.toDF().show()

+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  0|[112, 33, 165, 24...|
|  1|[91, 17, 15, 14, ...|
|  2|[150, 248, 88, 27...|
|  3|[46, 73, 0, 45, 1...|
|  4|[61, 77, 100, 389...|
|  5|[71, 42, 73, 325,...|
|  6|[72, 397, 131, 25...|
|  7|[10, 32, 37, 100,...|
|  8|[31, 22, 9, 85, 8...|
|  9|[27, 32, 33, 49, ...|
| 10|[114, 100, 41, 50...|
| 11|[7, 192, 153, 44,...|
| 12|[26, 120, 15, 29,...|
| 13|[10, 71, 9, 92, 7...|
| 14|[5, 135, 24, 20, ...|
| 15|[92, 22, 150, 226...|
| 16|[114, 39, 1, 38, ...|
| 17|[31, 60, 23, 11, ...|
| 18|[31, 5, 9, 169, 1...|
| 19|[63, 21, 156, 11,...|
+---+--------------------+
only showing top 20 rows



# LSH

In [ ]:
def map_buckets(row):

    band_nr = 5
    row_nr = 10
    doc_id = row[0]
    doc_sign = row[1]
    hash_funct = hashFamily(1)
    out = []

    for i in range(0,band_nr):
        band_id = i
        idx = i*row_nr
        set_col = ' '.join(str(x) for x in doc_sign[idx:idx+row_nr])
        bucket = hash_funct.get_hash_value(set_col)
        out.append( ( (band_id, bucket), doc_id)  )

    return iter(out) #since we are going to return multiple tuple we call flatMap() and return an iterator on those tuple


In [ ]:
candidates = sig_matrix.flatMap(map_buckets).groupByKey().map(lambda x : (x[0], list(x[1])))

In [ ]:
candidates.toDF().show()

+----------+---+
|        _1| _2|
+----------+---+
|{0, 19888}|[0]|
|{1, 30040}|[0]|
|{2, 63225}|[0]|
|{3, 49022}|[0]|
|{4, 21767}|[0]|
|{0, 51117}|[1]|
|{1, 20314}|[1]|
|{2, 60241}|[1]|
|{3, 64664}|[1]|
|{4, 60462}|[1]|
|{0, 43030}|[2]|
|{1, 46047}|[2]|
|{2, 21694}|[2]|
|{3, 24114}|[2]|
| {4, 8036}|[2]|
|{0, 52071}|[3]|
|{1, 12186}|[3]|
| {2, 8892}|[3]|
|{3, 59805}|[3]|
|{4, 37658}|[3]|
+----------+---+
only showing top 20 rows



In [ ]:
similar_pairs = set()
sig_df = sig_matrix
sig_df.persist() #store it in memory to avoid lazy behaviour
sig_df = sig_df.collect() #collect all the distributed record to speedup computation
start_time = time.time()

for candidates_list in candidates.map(lambda x: x[1]).collect(): #collecting all candidate lists from worker nodes

    candidates_nr = len(candidates_list)
    print(candidates_list)
    print(candidates_nr)

    for i in range(0,candidates_nr-1):
        for j in range(i+1,candidates_nr):

            doc_id_1 = candidates_list[i]
            doc_id_2 = candidates_list[j]

            #TEST PER VEDERE SE FA COPPIE (i, i)
            if (doc_id_1 == doc_id_2):
              same_doc_pair_checked += 1

            docId_Sig_1 = sig_df[doc_id_1]
            docId_Sig_2 = sig_df[doc_id_2]
            if docId_Sig_1[0] != doc_id_1:
                raise Exception("DocId is %d while retrived %d from signature matrix."%(doc_id_1,docId_Sig_1[0]))
            if docId_Sig_2[0] != doc_id_2:
                raise Exception("DocId is %d while retrived %d from signature matrix."%(doc_id_1,docId_Sig_2[0]))

            sig_1 = set(docId_Sig_1[1]) #transform signature into a set
            sig_2 = set(docId_Sig_2[1])
            #sig_1 = sig_df.filter("_1 =="+" "+ str(doc_id_1) ).select('_2') #get signature of doc 1
            #sig_2 = sig_df.filter("_1 =="+" "+ str(doc_id_2) ).select('_2') #get signature of doc 2

            #sig_2 = set(sig_2.collect()[0]['_2'])
            js = len(sig_1.intersection(sig_2) ) / len(sig_1.union(sig_2) ) #Compute Jaccard'Similarity
            if js >= 0.2:
                pair = tuple(sorted((doc_id_1,doc_id_2) ))
                similar_pairs.add(  pair   )

end_time = time.time()

lsh_time = end_time - start_time

print("FOUND %d SIMILAR PAIRS" %len(similar_pairs))

print("paia di stesso doc %d" %same_doc_pair_checked)

Output streaming troncato alle ultime 5000 righe.
[493]
1
[493]
1
[493]
1
[494]
1
[494]
1
[494]
1
[494]
1
[494]
1
[495]
1
[495]
1
[495]
1
[495]
1
[495]
1
[496]
1
[496]
1
[496]
1
[496]
1
[496]
1
[497]
1
[497]
1
[497]
1
[497]
1
[497]
1
[498]
1
[498]
1
[498]
1
[498]
1
[498]
1
[499]
1
[499]
1
[499]
1
[499]
1
[499]
1
[500]
1
[500, 604]
2
[500]
1
[500]
1
[500]
1
[501]
1
[501]
1
[501]
1
[501]
1
[501]
1
[502]
1
[502]
1
[502]
1
[502]
1
[502]
1
[503]
1
[503]
1
[503]
1
[503]
1
[503]
1
[504]
1
[504]
1
[504]
1
[504]
1
[504]
1
[505]
1
[505]
1
[505]
1
[505]
1
[505]
1
[506]
1
[506]
1
[506]
1
[506]
1
[506]
1
[507]
1
[507]
1
[507]
1
[507]
1
[507]
1
[508]
1
[508]
1
[508]
1
[508]
1
[508]
1
[509]
1
[509]
1
[509]
1
[509]
1
[509]
1
[510]
1
[510]
1
[510]
1
[510]
1
[510]
1
[511]
1
[511]
1
[511]
1
[511]
1
[511]
1
[512]
1
[512]
1
[512]
1
[512]
1
[512]
1
[513]
1
[513]
1
[513]
1
[513]
1
[513]
1
[514]
1
[514]
1
[514]
1
[514]
1
[514]
1
[515]
1
[515]
1
[515]
1
[515]
1
[515]
1
[516]
1
[516]
1
[516]
1
[516]
1
[516]
1
[

In [ ]:
similar_pairs

{(175, 220),
 (191, 601),
 (193, 340),
 (205, 417),
 (215, 409),
 (338, 732),
 (500, 604),
 (946, 949)}

In [ ]:
DocID_Shingles=docId_shingleset_hfunc.map(lambda x: (x[0],x[1]) ).reduceByKey(lambda x,y: x).collect()
#DocID_Shingles.persist()
doc_nr = len(DocID_Shingles)
similar_pairs_bf = set()

start_time = time.time()

same_doc_pair_checked = 0

for i in range(0,doc_nr-1):
    for j in range(i,doc_nr):
        doc_id_1 = DocID_Shingles[i][0] #DocID_Shingles[i] is  tuple (doc_id, shingle_set)
        doc_id_2 = DocID_Shingles[j][0]

        #TEST PER VEDERE SE FA COPPIE (i, i)
        if (doc_id_1 == doc_id_2):
          same_doc_pair_checked += 1

        sig_1 = set(DocID_Shingles[i][1]) #transform shingle_set into a python set data structure
        sig_2 = set(DocID_Shingles[j][1])
        js = len(sig_1.intersection(sig_2) ) / len(sig_1.union(sig_2) ) #Compute Jaccard'Similarity
        if js >= 0.2:
            pair = tuple(sorted((doc_id_1,doc_id_2) ))
            similar_pairs_bf.add(  pair   )

end_time = time.time()

bf_time = end_time - start_time

print("FOUND %d SIMILAR PAIRS WITH BRUTEFORCE" %len(similar_pairs_bf))

print("paia di stesso doc %d" %same_doc_pair_checked)

FOUND 1004 SIMILAR PAIRS WITH BRUTEFORCE
paia di stesso doc 999


In [ ]:
similar_pairs_bf

{(428, 428),
 (639, 639),
 (277, 277),
 (488, 488),
 (126, 126),
 (520, 520),
 (731, 731),
 (369, 369),
 (580, 580),
 (218, 218),
 (429, 429),
 (461, 461),
 (672, 672),
 (310, 310),
 (521, 521),
 (631, 631),
 (764, 764),
 (402, 402),
 (25, 25),
 (613, 613),
 (462, 462),
 (572, 572),
 (494, 494),
 (705, 705),
 (554, 554),
 (403, 403),
 (435, 435),
 (646, 646),
 (495, 495),
 (605, 605),
 (738, 738),
 (409, 410),
 (587, 587),
 (436, 436),
 (546, 546),
 (59, 59),
 (679, 679),
 (890, 890),
 (528, 528),
 (377, 377),
 (0, 0),
 (982, 982),
 (620, 620),
 (831, 831),
 (469, 469),
 (579, 579),
 (92, 92),
 (712, 712),
 (923, 923),
 (561, 561),
 (184, 184),
 (410, 410),
 (33, 33),
 (653, 653),
 (864, 864),
 (502, 502),
 (125, 125),
 (823, 823),
 (956, 956),
 (594, 594),
 (217, 217),
 (805, 805),
 (553, 553),
 (915, 915),
 (66, 66),
 (686, 686),
 (897, 897),
 (158, 158),
 (856, 856),
 (7, 7),
 (989, 989),
 (250, 250),
 (838, 838),
 (948, 948),
 (99, 99),
 (797, 797),
 (719, 719),
 (159, 159),
 (930,